# Bounded forecasts

In forecasting, we often want to make sure the predictions stay within a certain range. For example, for predicting the sales of a product, we may require all forecasts to be positive. Thus, the forecasts may need to be bounded.

With TimeGPT, you can create bounded forecasts by transforming your data prior to calling the forecast function.

## 1. Import packages
First, we install and import the required packages

In [ ]:
#| hide
from nixtla.utils import colab_badge

In [ ]:
#| echo: false
colab_badge('docs/tutorials/14_bounded_forecasts')

In [ ]:
#| hide
from itertools import product

from fastcore.test import test_eq, test_fail, test_warns
from dotenv import load_dotenv

In [ ]:
#| hide
load_dotenv()

In [ ]:
import pandas as pd
import numpy as np
import os

from nixtla import NixtlaClient

In [ ]:
nixtla_client = NixtlaClient(
    # defaults to os.environ.get("NIXTLA_API_KEY")
    api_key = 'my_api_key_provided_by_nixtla'
)

In [ ]:
#| hide
nixtla_client = NixtlaClient()

## 2. Load data

We use the [] dataset, which should be strictly positive, as we can't observe a negative amount of [].

In [ ]:
df = []

df.head(10)

## 3. Bounded forecasts

First, we transform the target data to lie within a specified range. In this case, we choose a range of [ , ].

In [ ]:
df_transformed = df.copy()
y_min = 0
y_max = 100
df_transformed['y'] = np.log((df_transformed['y'] - y_min) / (y_max - df_transformed['y']))

We can create the forecasts

In [ ]:
forecast = timegpt()

After having created the forecasts, we need to inverse the transformation that we applied earlier:

In [ ]:
df_transformed['TimeGPT-bounded'] = ((y_max - y_min) * np.exp(df_transformed['TimeGPT'])) / (1 + np.exp(np.exp(df_transformed['TimeGPT'])) + y_min

Now, we can plot the forecasts

In [ ]:
plot_code

We can compare the forecasts to the situation where we don't apply a transformation

In [ ]:
forecast = timegpt()

In [ ]:
plot_code

As you can see, transofmration works.